# Exploring the data downloaded from USDA FoodData Central

See the download here: https://fdc.nal.usda.gov/download-datasets.html

Data available in `.data/`.

Data dictionary available in  `nutrify/data_exploration/data/FoodData_Central_foundation_food_csv_2021-04-28/Download & API Field Descriptions April 2021.pdf`





In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from tqdm.auto import tqdm

from pathlib import Path

## Get Data

In [13]:
targ_dir = "data/2022/JSON"

# Get all the files in the directory
data_files = sorted(list(Path(targ_dir).glob("*.json")))
data_files

# Read in the first file
with open(data_files[0]) as f:
    data = json.load(f)

len(data["FoundationFoods"])

210

In [26]:
nutrify_to_fdcid = pd.read_csv("data/2023-03-04-nutrify-name-to-fdc-id.csv")
nutrify_to_fdcid.head()

# Turn the class_name and fdc_id into a dictionary
nutrify_to_fdcid_dict = dict(zip(nutrify_to_fdcid["class_name"], nutrify_to_fdcid["fdc_id"]))


nutrify_to_fdcid_dict = {k: None if pd.isna(v) else v for k, v in nutrify_to_fdcid_dict.items()}

# Reverse the dictionary
fdcid_to_nutrify_name = {v: k for k, v in nutrify_to_fdcid_dict.items()}
fdcid_to_nutrify_name
# nutrify_to_fdcid_dict

{None: 'tomato_chutney',
 2262074.0: 'almond_butter',
 2346393.0: 'almonds',
 168175.0: 'apple_custard',
 1750342.0: 'apple_green',
 1750339.0: 'apple_red',
 171697.0: 'apricot',
 169205.0: 'artichoke',
 168389.0: 'asparagus',
 171705.0: 'avocado',
 749420.0: 'bacon',
 2343212.0: 'bagel',
 2343527.0: 'baklava',
 1105314.0: 'banana',
 174523.0: 'barbecue_sauce',
 172232.0: 'basil',
 170917.0: 'bay_leaves',
 75101000.0: 'bean_sprouts',
 747444.0: 'beans',
 2341807.0: 'beef_curry',
 2341259.0: 'beef_mince',
 2341260.0: 'beef_patty',
 2341252.0: 'beef_roast',
 2342187.0: 'beef_stir_fry',
 2346192.0: 'beer',
 2345290.0: 'beetroot',
 2344767.0: 'blackberries',
 2346411.0: 'blueberries',
 2345294.0: 'bok_choy',
 335240.0: 'bread',
 2343090.0: 'bread_naan',
 747447.0: 'broccoli',
 2345073.0: 'broccolini',
 2343384.0: 'brownie',
 2345292.0: 'brussel_sprouts',
 2344031.0: 'burrito',
 790508.0: 'butter',
 2346407.0: 'cabbage_green',
 2346408.0: 'cabbage_red',
 2345417.0: 'cabbage_savoy',
 2621314

In [28]:
nutrify_to_fdcid_keys = [int(key) for key in fdcid_to_nutrify_name.keys() if key is not None]
print(nutrify_to_fdcid_keys[:10])

# Create list of nutrify food items
nutrify_food_items = []
for item in data["FoundationFoods"]:
    if item["fdcId"] in nutrify_to_fdcid_keys:
        # item["nutrify_name"] = fdcid_to_nutrify_name[item["fdcI
        nutrify_food_items.append(item)

len(nutrify_food_items)

[2262074, 2346393, 168175, 1750342, 1750339, 171697, 169205, 168389, 171705, 749420]


72

In [30]:
nutrify_food_items[0]

{'foodClass': 'FinalFood',
 'description': 'Tomatoes, grape, raw',
 'foodNutrients': [{'type': 'FoodNutrient',
   'id': 2219983,
   'nutrient': {'id': 1051,
    'number': '255',
    'name': 'Water',
    'rank': 100,
    'unitName': 'g'},
   'dataPoints': 18,
   'foodNutrientDerivation': {'code': 'A',
    'description': 'Analytical',
    'foodNutrientSource': {'id': 1,
     'code': '1',
     'description': 'Analytical or derived from analytical'}},
   'max': 93.6,
   'min': 90.9,
   'median': 92.7,
   'amount': 92.5},
  {'type': 'FoodNutrient',
   'id': 2219984,
   'nutrient': {'id': 1162,
    'number': '401',
    'name': 'Vitamin C, total ascorbic acid',
    'rank': 6300,
    'unitName': 'mg'},
   'dataPoints': 12,
   'foodNutrientDerivation': {'code': 'A',
    'description': 'Analytical',
    'foodNutrientSource': {'id': 1,
     'code': '1',
     'description': 'Analytical or derived from analytical'}},
   'max': 32.4,
   'min': 22.4,
   'median': 26.3,
   'amount': 27.2},
  {'type': 

In [2]:
# Import databases
food = pd.read_csv("data/2022/FoodData_Central_foundation_food_csv_2022-10-28/food.csv")
food_survey = pd.read_csv("data/2022/FoodData_Central_survey_food_csv_2022-10-28/food.csv")
nutrient = pd.read_csv("data/2022/FoodData_Central_Supporting_Data_csv_2022-10-28/nutrient.csv")
food_nutrient = pd.read_csv("data/2022/FoodData_Central_foundation_food_csv_2022-10-28/food_nutrient.csv")
food_nutrient_survey = pd.read_csv("data/2022/FoodData_Central_survey_food_csv_2022-10-28/food_nutrient.csv")

print(len(food), len(food_survey), len(nutrient), len(food_nutrient), len(food_nutrient_survey))

# Combine food and food_survey and drop columns that don't have a description 
food = pd.concat([food, food_survey]).dropna(subset=["description"])
food["description"] = food["description"].str.lower()
print(f"Combined food rows: {len(food)}")

# Combine food_nutrient and food_nutrient_survey
food_nutrient = pd.concat([food_nutrient, food_nutrient_survey])
food_nutrient["nutrient_name"] = food_nutrient["nutrient_id"].map(nutrient.set_index("id")["name"]).str.lower() 
print(f"Combined food nutrient rows: {len(food_nutrient)}")

50582 5624 474 130334 365560
Combined food rows: 56198
Combined food nutrient rows: 495894


/tmp/ipykernel_3897715/1100687683.py:11: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  food_nutrient = pd.read_csv("data/2022/FoodData_Central_foundation_food_csv_2022-10-28/food_nutrient.csv")


In [3]:
food_nutrient.columns

Index(['id', 'fdc_id', 'nutrient_id', 'amount', 'data_points', 'derivation_id',
       'min', 'max', 'median', 'footnote', 'min_year_acqured',
       'min_year_acquired', 'nutrient_name'],
      dtype='object')

In [4]:
len(food_nutrient)

495894

In [5]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"hummus, sabra classic",16.0,2019-04-01
1,319875,market_acquisition,"hummus, sabra classic",16.0,2019-04-01
2,319876,market_acquisition,"hummus, sabra classic",16.0,2019-04-01
3,319877,sub_sample_food,hummus,16.0,2019-04-01
4,319878,sub_sample_food,hummus,16.0,2019-04-01


In [6]:
food_nutrient.head()

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,min_year_acquired,nutrient_name
0,2201847,319877,1051,56.30,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,water
1,2201845,319877,1002,1.28,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,nitrogen
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
3,2201844,319877,1007,1.98,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,ash
4,2201852,319878,1091,188.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"phosphorus, p"


In [7]:
# How many unique?
unique_descriptions = food["description"].unique()
len(unique_descriptions)

17048

In [8]:
for item in unique_descriptions:
    if "rutabaga" in item:
        print(item)

rutabaga, raw
rutabaga, cooked


Beautiful, this gives us ~17048 foods to work with as a goal to model. But surely they can be split into less categories?

In [9]:
unique_descriptions[:10]

array(['hummus, sabra classic', 'hummus', 'hummus, other',
       'hummus - nfy12140o', 'hummus - nfy12140p', 'hummus - nfy12140q',
       'hummus - nfy12140r', 'hummus - nfy12140s', 'hummus - nfy12140f',
       'hummus - nfy12140g'], dtype=object)

Where do these descriptions come from?

How can we reduce them down to like 10 unique foods and keep it simple...

In [10]:
unique_descriptions[-10:]

array(['cauliflower, cooked, as ingredient',
       'eggplant, cooked, as ingredient',
       'green beans, cooked, as ingredient',
       'summer squash, cooked, as ingredient',
       'dark green vegetables as ingredient in omelet',
       'tomatoes as ingredient in omelet',
       'other vegetables as ingredient in omelet',
       'vegetables as ingredient in curry',
       'sauce as ingredient in hamburgers',
       'industrial oil as ingredient in food'], dtype=object)

In [11]:
# Find random indexes of food to explore
import random
random_number = random.randint(0, len(unique_descriptions)-10)
unique_descriptions[random_number:random_number+10]

array(['proximates, haddock, raw (ct) - nfy060cmn',
       'haddock, raw (co) - cy060qt',
       'haddock, raw, iceland, kroger haddock fillets (co) - nfy060def',
       'haddock, raw, iceland, kroger haddock fillets (co) - nfy060dee',
       'minerals, haddock, raw (co) - nfy060cm2',
       'selenium, haddock, raw (co) - nfy060cm3',
       'fatty acids, haddock, raw (co) - nfy060cm7',
       'proximates, haddock, raw (co) - nfy060cm8',
       'haddock, raw (in) - cy060qv',
       'haddock, raw, imported from china, kroger haddock fillets (in) - nfy060d7w'],
      dtype=object)

### Food Categories

Let's dive into food categories. 

In [12]:
food.head()

,fdc_id,data_type,description,food_category_id,publication_date
0,319874,sample_food,"hummus, sabra classic",16.0,2019-04-01
1,319875,market_acquisition,"hummus, sabra classic",16.0,2019-04-01
2,319876,market_acquisition,"hummus, sabra classic",16.0,2019-04-01
3,319877,sub_sample_food,hummus,16.0,2019-04-01
4,319878,sub_sample_food,hummus,16.0,2019-04-01


In [13]:
food.data_type.value_counts()

sub_sample_food             40591
market_acquisition           6195
survey_fndds_food            5624
sample_food                  2697
agricultural_acquisition      810
foundation_food               281
Name: data_type, dtype: int64

In [15]:
# Find foods with "squash" in the description
food[food["description"].str.contains("squash")]

,fdc_id,data_type,description,food_category_id,publication_date
4390,2345150,survey_fndds_food,"sweet potato, squash, pumpkin, chrysanthemum, ...",NaN,2022-10-28
4446,2345206,survey_fndds_food,"winter squash, raw",NaN,2022-10-28
4447,2345207,survey_fndds_food,"winter squash, cooked, no added fat",NaN,2022-10-28
4448,2345208,survey_fndds_food,"winter squash, cooked, fat added",NaN,2022-10-28
4449,2345209,survey_fndds_food,"squash, winter, souffle",NaN,2022-10-28
4471,2345231,survey_fndds_food,"squash, winter type, soup, home recipe, canned...",NaN,2022-10-28
4567,2345327,survey_fndds_food,"summer squash, yellow, raw",NaN,2022-10-28
4568,2345328,survey_fndds_food,"summer squash, green, raw",NaN,2022-10-28
4694,2345454,survey_fndds_food,"flowers or blossoms of sesbania, squash, or li...",NaN,2022-10-28
4755,2345515,survey_fndds_food,"summer squash, yellow or green, fresh, cooked,...",NaN,2022-10-28


In [16]:
foundation_survey_and_sample_foods = food[food["data_type"].isin(["foundation_food", "sample_food", "survey_fndds_food"])].drop_duplicates(subset="description")
print(len(foundation_survey_and_sample_foods))
foundation_survey_and_sample_foods.head()
foundation_survey_and_sample_foods_list = foundation_survey_and_sample_foods["description"].tolist()

6895


In [17]:
foundation_and_sample_foods = food[food["data_type"].isin(["foundation_food", "sample_food"])].drop_duplicates(subset="description")
print(len(foundation_and_sample_foods))
foundation_and_sample_foods.head()
foundation_and_sample_foods_list = foundation_and_sample_foods["description"].tolist()

1289


In [18]:
foundation_food = food[food.data_type == "foundation_food"].sort_values("description", ascending=True).drop_duplicates(subset="description")
print(len(foundation_food))
foundation_foods_list = sorted(foundation_food.description.to_list())
print(len(foundation_foods_list))

214
214


In [19]:
# foundation_food.to_csv("data/2023-03-04-foundation_food.csv", index=False)

In [20]:
foundation_foods_list

['almond butter, creamy',
 'almond milk, unsweetened, plain, refrigerated',
 'almond milk, unsweetened, plain, shelf stable',
 'apple juice, with added vitamin c, from concentrate, shelf stable',
 'apples, fuji, with skin, raw',
 'apples, gala, with skin, raw',
 'apples, granny smith, with skin, raw',
 'apples, honeycrisp, with skin, raw',
 'apples, red delicious, with skin, raw',
 'applesauce, unsweetened, with added vitamin c',
 'bananas, overripe, raw',
 'bananas, ripe and slightly ripe, raw',
 'beans, dry, black (0% moisture)',
 'beans, dry, brown (0% moisture)',
 'beans, dry, carioca (0% moisture)',
 'beans, dry, cranberry (0% moisture)',
 'beans, dry, dark red kidney (0% moisture)',
 'beans, dry, flor de mayo (0% moisture)',
 'beans, dry, great northern (0% moisture)',
 'beans, dry, light red kidney (0% moisture)',
 'beans, dry, light tan (0% moisture)',
 'beans, dry, medium red (0% moisture)',
 'beans, dry, navy (0% moisture)',
 'beans, dry, pink (0% moisture)',
 'beans, dry, pi

In [21]:
# Create a function to similarity match the class names (e.g. code which string is most like another string)
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import dot_score
model = SentenceTransformer("all-MiniLM-L6-v2")

def embed_list_of_classes(class_names: list, model: SentenceTransformer):
    """
    Embeds a list of class names.
    """

    # Map the class_name to the embedding
    class_name_to_embedding = {class_name: embedding for class_name, embedding in zip(class_names, model.encode(class_names))}

    return class_name_to_embedding

# Create a function to similarity match the class names (e.g. code which string is most like another string)
def find_most_similar_class_name(target_class_name, class_name_embedding_dict, num_similar=3):
    """
    Finds the most similar class name to the class_name provided.
    """
    # Get the embedding of the target_class_name
    target_class_name_embedding = model.encode([target_class_name])[0]

    # Create a dictionary of the class_name and the dot score between the target_class_name and the class_name
    class_name_embedding_dict = {class_name: dot_score(embedding, target_class_name_embedding) for class_name, embedding in class_name_embedding_dict.items()}

    # Sort the dictionary by the dot score
    most_similar_class_names = {k: v[0].item() for k, v in sorted(class_name_embedding_dict.items(), key=lambda item: item[1], reverse=True)}

    return most_similar_class_names

In [22]:
# Find the most similar class names to X
class_name_embedding_dict_unique_descriptions = embed_list_of_classes(unique_descriptions, model)

class_name_embedding_dict_foundation_foods = embed_list_of_classes(foundation_foods_list, model)

class_name_embedding_dict_foundation_and_sample_foods = embed_list_of_classes(foundation_and_sample_foods_list, model)

class_name_embedding_dict_foundation_survey_and_sample_foods = embed_list_of_classes(foundation_survey_and_sample_foods_list, model)

# class_name_embedding_dict 

In [139]:
most_similar_class_names = find_most_similar_class_name("zucchini", class_name_embedding_dict_foundation_survey_and_sample_foods)
most_similar_class_names

{'bread, zucchini': 0.8592194318771362,
 'zucchini, pickled': 0.8471095561981201,
 'muffin, zucchini': 0.815552830696106,
 'cake or cupcake, zucchini': 0.7447828054428101,
 'gnocchi, potato': 0.7159714698791504,
 'gnocchi, cheese': 0.6733115911483765,
 'spaghetti squash, cooked': 0.5928136110305786,
 'fruit, nfs': 0.5418483018875122,
 'bread, vegetable': 0.5338558554649353,
 'squash, winter, souffle': 0.531885027885437,
 'baby toddler vegetable, nfs': 0.5317342281341553,
 'cabbage with ham hocks': 0.5291968584060669,
 'winter squash, cooked, fat added': 0.5240758657455444,
 'potato, baked, peel eaten, with vegetables': 0.5167760252952576,
 'pasta, vegetable, cooked': 0.5157829523086548,
 'winter squash, raw': 0.5147427916526794,
 'mushroom, crimini': 0.5141078233718872,
 'squash, summer, souffle': 0.5125963091850281,
 'summer squash, cooked, as ingredient': 0.5124391317367554,
 'fried summer squash, yellow or green': 0.51114821434021,
 'squash, winter type, soup, home recipe, canned, o

In [129]:
list(most_similar_class_names.keys())[:10]

['apples, red delicious, with skin, raw',
 'apples, gala, with skin, raw',
 'apples, honeycrisp, with skin, raw',
 'apples, granny smith, with skin, raw',
 'applesauce, unsweetened, with added vitamin c',
 'apples, fuji, with skin, raw',
 'cherries, sweet, dark red, raw',
 'peaches, yellow, raw',
 'blueberries, raw',
 'onions, red, raw']

In [192]:
import openai

openai.api_key_path = "../foodvision/utils/openai_api_key.txt"

def chat_complete(prompt: str, **openai_kwargs) -> str:
    """Generate completion from OpenAI ChatGPT API"""
    default_kwargs = {"model": "gpt-3.5-turbo", "max_tokens": 256, "temperature": 0.2, "top_p": 1}
    openai_kwargs = {**default_kwargs, **openai_kwargs}
    response = openai.ChatCompletion.create(
        **openai_kwargs,
        messages=[
                {"role": "system", "content": "You are a helpful assistant who is very good at designing apps and making them look good."},
                {"role": "user", "content": prompt},
            ]
        )
    return response

base_prompt_for_sorting_food_names = """You are a food scientist and nutritionist very knowledgable on all different kinds of foods.\n
Given the following list of foods and the target text, please sort the list of foods into which ones relate most to the target text.\n
For example, the list ['apples, red delicious, with skin, raw',
 'apples, gala, with skin, raw',
 'apples, honeycrisp, with skin, raw',
 'apples, granny smith, with skin, raw',
 'applesauce, unsweetened, with added vitamin c',
 ...]
and the target text 'apple_red' would be sorted as a JSON dictionary,
 '1': 'apples, red delicious, with skin, raw', 
 '2': 'apples, gala, with skin, raw', 
 '3': 'apples, honeycrisp, with skin, raw', 
 '4': 'apples, granny smith, with skin, raw', 
 '5': 'applesauce, unsweetened, with added vitamin c', 
 ....\n
Please do not include any extra foods or text, only return the list of foods in the target food list.\n
Put a favour on similar types of foods, e.g. if the target text contains "capsicum_green" then "pepper, red" should be higher than "cabbage, green".\n
If the target list of foods to sort has N items, you should return a list of N items.\n
For example, if the target list of foods to sort has 10 items, you should return a list of 10 items.\n
Target text: {target_text}\n
List of foods to sort: {target_food_list}\n
Sorted list of food as properly formatted JSON dictionary:\n
"""

def sort_list_of_foods(base_prompt: str, target_text: str, target_food_list: list) -> list:
    # print(base_prompt)
    target_text_dict = {"target_text": target_text, "target_food_list": target_food_list}
    target_text_prompt = base_prompt.format(**target_text_dict)
    # target_text_prompt = base_prompt.format(target_text, target_food_list)
    # print(target_text_prompt)
    answer = chat_complete(target_text_prompt)
    answer_formatted = str(answer.choices[0].message.content)
    # Remove "\n"
    # print(f"Display name generated: {target_text} -> {answer_formatted}")
    return answer_formatted

target_text = "capsicum_red"

most_similar_class_names = find_most_similar_class_name(target_text, class_name_embedding_dict)

# Get the top 10 most similar class names
target_food_list = list(most_similar_class_names.keys())[:10]
print(len(target_food_list))

# Turn target_food_list into a dictionary with keys in order of 1, 2, 3...
target_food_list = {i+1: food_name for i, food_name in enumerate(target_food_list)}
print(target_food_list)
# print(target_food_list)

sorted_list_of_foods = sort_list_of_foods(base_prompt=base_prompt_for_sorting_food_names, target_text=target_text, target_food_list=target_food_list)
sorted_list_of_foods


10
{1: 'peppers, bell, red, raw', 2: 'lettuce, leaf, red, raw', 3: 'mustard, prepared, yellow', 4: 'peppers, bell, yellow, raw', 5: 'cabbage, red, raw', 6: 'cherries, sweet, dark red, raw', 7: 'peppers, bell, green, raw', 8: 'peppers, bell, orange, raw', 9: 'onions, red, raw', 10: 'beans, dry, medium red (0% moisture)'}


'{\n    "1": "peppers, bell, red, raw",\n    "2": "cabbage, red, raw",\n    "3": "cherries, sweet, dark red, raw",\n    "4": "lettuce, leaf, red, raw",\n    "5": "onions, red, raw",\n    "6": "peppers, bell, orange, raw",\n    "7": "peppers, bell, yellow, raw",\n    "8": "peppers, bell, green, raw",\n    "9": "mustard, prepared, yellow",\n    "10": "beans, dry, medium red (0% moisture)"\n}'

In [193]:
import json
sorted_list_of_foods = json.loads(sorted_list_of_foods)
sorted_list_of_foods

{'1': 'peppers, bell, red, raw',
 '2': 'cabbage, red, raw',
 '3': 'cherries, sweet, dark red, raw',
 '4': 'lettuce, leaf, red, raw',
 '5': 'onions, red, raw',
 '6': 'peppers, bell, orange, raw',
 '7': 'peppers, bell, yellow, raw',
 '8': 'peppers, bell, green, raw',
 '9': 'mustard, prepared, yellow',
 '10': 'beans, dry, medium red (0% moisture)'}

In [20]:
unique_categories = food["food_category_id"].unique()
len(unique_categories)

19

19 different food categories... I wonder what these are?

In [21]:
food["food_category_id"].value_counts()

11.0    10819
1.0      9057
9.0      8558
16.0     4575
4.0      2892
14.0     2889
12.0     2769
20.0     1949
5.0      1503
15.0      913
7.0       795
10.0      613
6.0       568
18.0      488
25.0      474
13.0      454
2.0       386
19.0       54
Name: food_category_id, dtype: int64

In [23]:
# Get food categories
food_cats = pd.read_csv("data/2022/FoodData_Central_Supporting_Data_csv_2022-10-28/food_category.csv")
food_cats

,id,code,description
0,1,100,Dairy and Egg Products
1,2,200,Spices and Herbs
2,3,300,Baby Foods
3,4,400,Fats and Oils
4,5,500,Poultry Products
5,6,600,"Soups, Sauces, and Gravies"
6,7,700,Sausages and Luncheon Meats
7,8,800,Breakfast Cereals
8,9,900,Fruits and Fruit Juices
9,10,1000,Pork Products


## 10 foods we want

To keep things simple, we will reduce the databases from FoodData Central to 10 different foods.

Why these foods?

Because we have images for those foods ready to go.

```python
# These aren't whole foods so we don't want them yet, let's get another list and get those
ten_foods = ["chicken_curry", 
"chicken_wings", 
"fried_rice", 
"grilled_salmon", 
"humburger", 
"ice_cream", 
"pizza",
"ramen", 
"steak", 
"sushi"]

# We want these... (they're whole foods) 
ten_whole_foods = ["chicken_wings",
    "apple",
    "banana",
    "beef", # steak, etc
    "carrots",
    "egg", # whole egg
    "strawberries",
    "blueberries",
    "mushrooms",
    "honey"
]
```

In [24]:
ten_whole_foods = ['apple',
 'banana',
 'beef', # steak etc
 'blueberries',
 'carrots',
 'chicken_wings',
 'egg', # whole egg
 'honey',
 'mushrooms',
 'strawberries']
ten_whole_foods

['apple',
 'banana',
 'beef',
 'blueberries',
 'carrots',
 'chicken_wings',
 'egg',
 'honey',
 'mushrooms',
 'strawberries']

In [61]:
food.head()

AttributeError: 'str' object has no attribute 'head'

In [60]:
food.data_type.value_counts()

AttributeError: 'str' object has no attribute 'data_type'

In [26]:
# Foundation food is the ground truth for a certain type of food, excludes some details about the food
# E.g. the data_type foundation_food for Chicken will the the original unique ID for chicken
foundation_food = food[(food["data_type"] == "foundation_food") | (food["data_type"] == "survey_fndds_food")]
len(foundation_food)

5905

survey_fndds_food    5624
foundation_food       281
Name: data_type, dtype: int64

In [27]:
foundation_food[foundation_food["description"].str.contains("blue")]

,fdc_id,data_type,description,food_category_id,publication_date
42186,2263889,foundation_food,"blueberries, raw",9.0,2022-04-28
43441,2346411,foundation_food,"blueberries, raw",9.0,2022-10-28
348,2341108,survey_fndds_food,"cheese, blue or roquefort",NaN,2022-10-28
2720,2343480,survey_fndds_food,"pie, blueberry",NaN,2022-10-28
3933,2344693,survey_fndds_food,"blueberries, dried",NaN,2022-10-28
4009,2344769,survey_fndds_food,"blueberries, raw",NaN,2022-10-28
4011,2344771,survey_fndds_food,"blueberries, frozen",NaN,2022-10-28
4012,2344772,survey_fndds_food,blueberry pie filling,NaN,2022-10-28
4057,2344817,survey_fndds_food,blueberry juice,NaN,2022-10-28
4994,2345754,survey_fndds_food,blue or roquefort cheese dressing,NaN,2022-10-28


In [29]:
foundation_foods = foundation_food["description"]
foundation_foods.iloc[20:40]

4153               peanut butter, smooth style, with salt
4329                             cheese, parmesan, grated
4491    cheese, pasteurized process, american, vitamin...
4580    grapefruit juice, white, canned or bottled, un...
4723                                 peaches, yellow, raw
4817    seeds, sunflower seed kernels, dry roasted, wi...
4951      sausage, italian, pork, mild, cooked, pan-fried
5164                  bread, white, commercially prepared
5285          sausage, turkey, breakfast links, mild, raw
5428                                        cheese, swiss
5489    kale, frozen, cooked, boiled, drained, without...
5751    carrots, frozen, unprepared (includes foods fo...
5991                            mustard, prepared, yellow
6198                                figs, dried, uncooked
6339                                kiwifruit, green, raw
6491                              melons, cantaloupe, raw
6650                                      nectarines, raw
6794    orange

In [30]:
# Found a list of the foundation foods we're going to start with!
foundation_foods_list = list(foundation_foods)
for food in foundation_foods_list:
    if "blue" in food:
        print(food)

blueberries, raw
blueberries, raw
cheese, blue or roquefort
pie, blueberry
blueberries, dried
blueberries, raw
blueberries, frozen
blueberry pie filling
blueberry juice
blue or roquefort cheese dressing
blue or roquefort cheese dressing, light
blue or roquefort cheese dressing, fat free
blueberry syrup


In [31]:
# food.loc[(food["description"].str.contains("chicken", case=False)) & (food["description"].str.contains("drumstick", case=False))][-10:]
# Find chicken in foundation food
for food in foundation_foods:
    if "chicken" in food.lower():
        print(food)

chicken, broilers or fryers, drumstick, meat only, cooked, braised
chicken, broiler or fryers, breast, skinless, boneless, meat only, cooked, braised
mock chicken legs
chicken, ns as to part and cooking method, ns as to skin eaten
chicken, ns as to part and cooking method, skin eaten
chicken, ns as to part and cooking method, skin not eaten
chicken, ns as to part, baked, broiled, or roasted, ns as to skin eaten
chicken, ns as to part, baked, broiled, or roasted, skin eaten
chicken, ns as to part, baked, broiled, or roasted, skin not eaten
chicken, ns as to part, rotisserie, ns as to skin eaten
chicken, ns as to part, rotisserie, skin eaten
chicken, ns as to part, rotisserie, skin not eaten
chicken, ns as to part, stewed, ns as to skin eaten
chicken, ns as to part, stewed, skin eaten
chicken, ns as to part, stewed, skin not eaten
chicken, ns as to part, grilled without sauce, ns as to skin eaten
chicken, ns as to part, grilled without sauce, skin eaten
chicken, ns as to part, grilled wi

In [32]:
chicken_wing_id = int(foundation_food.loc[foundation_food["description"].str.contains("Chicken", case=False)].iloc[0]["fdc_id"])
chicken_wing_id

331897

In [33]:
food_nutrient[food_nutrient["fdc_id"] == chicken_wing_id]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,min_year_acquired,nutrient_name
41650,2259068,331897,1303,0.003,5.0,1.0,0.002,0.004,0.003,NaN,2010.0,NaN,tfa 16:1 t
41651,2259065,331897,1280,0.008,5.0,1.0,0.008,0.009,0.008,NaN,2010.0,NaN,pufa 22:5 n-3 (dpa)
41652,2259076,331897,1404,0.045,5.0,1.0,0.035,0.059,0.042,NaN,2010.0,NaN,"pufa 18:3 n-3 c,c,c (ala)"
41653,2259059,331897,1261,0.002,5.0,1.0,0.001,0.003,0.002,NaN,2010.0,NaN,sfa 8:0
41654,2259106,331897,1109,0.170,1.0,1.0,NaN,NaN,0.170,NaN,2010.0,NaN,vitamin e (alpha-tocopherol)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41740,2259066,331897,1300,0.006,5.0,1.0,0.005,0.007,0.006,NaN,2010.0,NaN,sfa 17:0
41741,2259121,331897,1271,0.088,5.0,1.0,0.083,0.094,0.087,NaN,2010.0,NaN,pufa 20:4
41742,2259112,331897,1167,5.050,5.0,1.0,4.890,5.240,5.050,NaN,2010.0,NaN,niacin
41743,2259074,331897,1329,0.021,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,"fatty acids, total trans-monoenoic"


## Get protein, carb, fat IDs

See this document for info on foundation foods and their nutrients - https://fdc.nal.usda.gov/docs/Foundation_Foods_Documentation_Apr2021.pdf

* Carbohydrate, by difference = total carbohydrates


In [34]:
nutrient[(nutrient["name"].str.contains("protein", case=False)) | \
         (nutrient["name"].str.contains("carbohydrate", case=False)) | \
         (nutrient["name"].str.contains("fat", case=False))]

,id,name,unit_name,nutrient_nbr,rank
4,1003,Protein,G,203.0,600.0
5,1004,Total lipid (fat),G,204.0,800.0
6,1005,"Carbohydrate, by difference",G,205.0,1110.0
50,1049,"Solids, non-fat",G,253.0,999999.0
51,1050,"Carbohydrate, by summation",G,205.2,1120.0
54,1053,Adjusted Protein,G,257.0,700.0
73,1072,"Carbohydrate, other",G,284.0,NaN
86,1085,Total fat (NLEA),G,298.0,900.0
258,1257,"Fatty acids, total trans",G,605.0,15400.0
259,1258,"Fatty acids, total saturated",G,606.0,9700.0


In [35]:
target_nutrients = nutrient[nutrient["name"].isin(["Protein", "Total lipid (fat)", "Carbohydrate, by difference"])]
target_nutrients

,id,name,unit_name,nutrient_nbr,rank
4,1003,Protein,G,203.0,600.0
5,1004,Total lipid (fat),G,204.0,800.0
6,1005,"Carbohydrate, by difference",G,205.0,1110.0


In [36]:
target_nutrient_dict = {1003: "protein",
    1004: "fat",
    1005: "carbohydrate"
}

## Get target food protein, fat, carbohydrates

We want to now index on the target foods and the target nutrients and retrieve their values for each food/nutrient.

E.g.

```python
{"food_1": {"protein": 100,
            "carbohydrate": 50,
            "fat": 20},
 "food_2": ...

...}
```

In [37]:
list(target_nutrient_dict.keys())

[1003, 1004, 1005]

In [38]:
food_nutrient

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,min_year_acquired,nutrient_name
0,2201847,319877,1051,56.300,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,water
1,2201845,319877,1002,1.280,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,nitrogen
2,2201846,319877,1004,19.000,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
3,2201844,319877,1007,1.980,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,ash
4,2201852,319878,1091,188.000,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,"phosphorus, p"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
365555,28906044,2346383,334,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365556,28906068,2346383,612,0.048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365557,28906042,2346383,323,10.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365558,28906052,2346383,417,0.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
food_nutrient[(food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,min_year_acquired,nutrient_name
2,2201846,319877,1004,19.00,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
16,2201859,319882,1004,18.70,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
28,2201873,319892,1004,16.60,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
43,2201886,319899,1004,19.10,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
97,2201942,319908,1004,18.20,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
...,...,...,...,...,...,...,...,...,...,...,...,...,...
129408,28911795,2352627,1004,0.12,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
129433,28911820,2352652,1004,0.19,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
129458,28911845,2352677,1004,0.17,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)
129485,28911872,2352704,1004,0.18,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,total lipid (fat)


In [40]:
food_nutrient.dtypes

id                     int64
fdc_id                 int64
nutrient_id            int64
amount               float64
data_points          float64
derivation_id        float64
min                  float64
max                  float64
median               float64
footnote              object
min_year_acqured     float64
min_year_acquired    float64
nutrient_name         object
dtype: object

In [41]:
# Find nutrition for chicken_wing_id (protein, fat, carb)
food_nutrient[(food_nutrient["fdc_id"] == chicken_wing_id) & (food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))]

,id,fdc_id,nutrient_id,amount,data_points,derivation_id,min,max,median,footnote,min_year_acqured,min_year_acquired,nutrient_name
41686,2259098,331897,1004,5.95,6.0,1.0,5.54,6.33,5.93,NaN,2010.0,NaN,total lipid (fat)
41718,2259079,331897,1003,23.90,NaN,49.0,23.00,24.60,24.10,NaN,NaN,NaN,protein
41729,2259099,331897,1005,0.00,NaN,49.0,NaN,NaN,NaN,NaN,NaN,NaN,"carbohydrate, by difference"


In [42]:
sorted(list(foundation_foods))

['abalone',
 'adobo, with noodles',
 'adobo, with rice',
 'agave liquid sweetener',
 'alcoholic malt beverage',
 'alcoholic malt beverage, sweetened',
 'alexander',
 'alfalfa sprouts, raw',
 'alfredo sauce',
 'alfredo sauce with added vegetables',
 'alfredo sauce with meat',
 'alfredo sauce with meat and added vegetables',
 'alfredo sauce with poultry',
 'alfredo sauce with poultry and added vegetables',
 'alfredo sauce with seafood',
 'alfredo sauce with seafood and added vegetables',
 'almond butter',
 'almond butter and jelly sandwich, on wheat bread',
 'almond butter and jelly sandwich, on white bread',
 'almond butter sandwich, on wheat bread',
 'almond butter sandwich, on white bread',
 'almond butter, creamy',
 'almond butter, lower sodium',
 'almond chicken',
 'almond milk, sweetened',
 'almond milk, sweetened, chocolate',
 'almond milk, unsweetened',
 'almond milk, unsweetened, chocolate',
 'almond milk, unsweetened, plain, refrigerated',
 'almond milk, unsweetened, plain, she

In [43]:
ten_whole_foods = ["chicken_wings",
    "apple",
    "banana",
    "beef", # steak, etc
    "carrots",
    "egg", # whole egg
    "strawberries",
    "blueberries",
    "mushrooms",
    "honey"
]

In [44]:
ten_whole_foods

['chicken_wings',
 'apple',
 'banana',
 'beef',
 'carrots',
 'egg',
 'strawberries',
 'blueberries',
 'mushrooms',
 'honey']

## Get ten whole foods `food_id`

Everything except blueberries and honey are available in `foundation_food`. 

For blueberries and honey, we'll have to dig into the survery data: `data_exploration/data/FoodData_Central_survey_food_csv_2020-10-30`

In [45]:
# Get all food ids from foundation_food (honey and blueberries in another dataset)
target_whole_foods = ['apple', # removed chicken wings... can come back later...
 'banana',
 'beef',
 'blueberries',
 'carrots',
 'chicken',
 'egg',
 'honey',
 'strawberries',
 'mushrooms']

In [46]:
# str.contains can search on regex - https://stackoverflow.com/a/17973255/7900723
pattern = "|".join([f"(?i){food}" for food in target_whole_foods])
pattern

'(?i)apple|(?i)banana|(?i)beef|(?i)blueberries|(?i)carrots|(?i)chicken|(?i)egg|(?i)honey|(?i)strawberries|(?i)mushrooms'

In [47]:
foundation_food[foundation_food["description"].str.contains(pattern, case=False)].sort_values(by=["description"])

,fdc_id,data_type,description,food_category_id,publication_date
1481,2342241,survey_fndds_food,almond chicken,NaN,2022-10-28
2208,2342968,survey_fndds_food,"almonds, honey roasted",NaN,2022-10-28
4053,2344813,survey_fndds_food,apple cider,NaN,2022-10-28
5405,2346165,survey_fndds_food,"apple juice beverage, 40-50% juice, light",NaN,2022-10-28
4054,2344814,survey_fndds_food,"apple juice, 100%",NaN,2022-10-28
...,...,...,...,...,...
1338,2342098,survey_fndds_food,"venison or deer, noodles, and vegetables inclu...",NaN,2022-10-28
1337,2342097,survey_fndds_food,"venison or deer, potatoes, and vegetables excl...",NaN,2022-10-28
1336,2342096,survey_fndds_food,"venison or deer, potatoes, and vegetables incl...",NaN,2022-10-28
2248,2343008,survey_fndds_food,"walnuts, excluding honey roasted",NaN,2022-10-28


In [48]:
foundation_food[foundation_food["description"].str.contains("honey")]

,fdc_id,data_type,description,food_category_id,publication_date
20191,1105547,foundation_food,"apples, honeycrisp, with skin, raw",9.0,2020-10-30
20547,1750343,foundation_food,"apples, honeycrisp, with skin, raw",9.0,2020-10-30
2208,2342968,survey_fndds_food,"almonds, honey roasted",NaN,2022-10-28
2215,2342975,survey_fndds_food,"cashews, honey roasted",NaN,2022-10-28
2228,2342988,survey_fndds_food,"mixed nuts, honey roasted",NaN,2022-10-28
2237,2342997,survey_fndds_food,"peanuts, honey roasted",NaN,2022-10-28
2242,2343002,survey_fndds_food,"pecans, honey roasted",NaN,2022-10-28
2248,2343008,survey_fndds_food,"walnuts, excluding honey roasted",NaN,2022-10-28
2249,2343009,survey_fndds_food,"walnuts, honey roasted",NaN,2022-10-28
2757,2343517,survey_fndds_food,"sopaipilla, without syrup or honey",NaN,2022-10-28


In [49]:
# Found this earlier
chicken_wing_id

331897

In [50]:
# Map foods to food_id (these have been filtered from larger quantities to smaller quantities)
# For example, if there were 5 kinds of apple, only one was chosen
whole_foods_id_map = {1750339: "apple", # red delicious
    1105314: "banana", # Bananas, ripe and slightly ripe, raw
    1102702: "blueberries", # blueberries, raw	
    746763: "beef", # t-bone steak 
    746764: "carrots", # frozen unprepared
    331897: "chicken_wings", # Chicken, broilers or fryers, drumstick, meat o...	
    329490: "egg", # Egg, whole, dried	
    1103956: "honey", # Honey
    1750347: "mushrooms", # Mushrooms, white button
    747448: "strawberries" # strawberries, raw
}

In [51]:
list(whole_foods_id_map.keys())

[1750339,
 1105314,
 1102702,
 746763,
 746764,
 331897,
 329490,
 1103956,
 1750347,
 747448]

In [52]:
# Find nutrition for eight whole foods
target_whole_foods_df = food_nutrient[(food_nutrient["fdc_id"].isin(list(whole_foods_id_map.keys()))) & \
    (food_nutrient["nutrient_id"].isin(list(target_nutrient_dict.keys())))][["fdc_id", "nutrient_id", "amount"]]
target_whole_foods_df

,fdc_id,nutrient_id,amount
34265,329490,1004,39.800000
34266,329490,1005,1.870000
34270,329490,1003,48.100000
41686,331897,1004,5.950000
41718,331897,1003,23.900000
41729,331897,1005,0.000000
71052,746763,1003,27.300000
71079,746763,1005,0.000000
71097,746763,1004,11.400000
71175,746764,1004,0.470000


In [53]:
# Pivot the table to how we want it
target_whole_foods_df = target_whole_foods_df.pivot_table("amount", "fdc_id", "nutrient_id")
target_whole_foods_df

nutrient_id,1003,1004,1005
fdc_id,,,
329490,48.100000,39.8000,1.870000
331897,23.900000,5.9500,0.000000
746763,27.300000,11.4000,0.000000
746764,0.810000,0.4700,7.920000
747448,0.640000,0.2200,7.630000
1105314,0.740000,0.2900,23.000000
1750339,0.187500,0.2125,14.781700
1750347,2.890625,0.3708,4.079375


In [54]:
len(whole_foods_id_map)

10

In [55]:
target_whole_foods_df = target_whole_foods_df.reset_index(drop=False).rename_axis(None, axis=1)
target_whole_foods_df

,fdc_id,1003,1004,1005
0,329490,48.100000,39.8000,1.870000
1,331897,23.900000,5.9500,0.000000
2,746763,27.300000,11.4000,0.000000
3,746764,0.810000,0.4700,7.920000
4,747448,0.640000,0.2200,7.630000
5,1105314,0.740000,0.2900,23.000000
6,1750339,0.187500,0.2125,14.781700
7,1750347,2.890625,0.3708,4.079375


In [56]:
target_nutrient_dict

{1003: 'protein', 1004: 'fat', 1005: 'carbohydrate'}

In [57]:
# Rename columns
target_whole_foods_df.rename(columns=target_nutrient_dict, inplace=True)
target_whole_foods_df

,fdc_id,protein,fat,carbohydrate
0,329490,48.100000,39.8000,1.870000
1,331897,23.900000,5.9500,0.000000
2,746763,27.300000,11.4000,0.000000
3,746764,0.810000,0.4700,7.920000
4,747448,0.640000,0.2200,7.630000
5,1105314,0.740000,0.2900,23.000000
6,1750339,0.187500,0.2125,14.781700
7,1750347,2.890625,0.3708,4.079375


In [58]:
# Add food names
target_whole_foods_df["food_name"] = target_whole_foods_df["fdc_id"].map(whole_foods_id_map)
target_whole_foods_df

,fdc_id,protein,fat,carbohydrate,food_name
0,329490,48.100000,39.8000,1.870000,egg
1,331897,23.900000,5.9500,0.000000,chicken_wings
2,746763,27.300000,11.4000,0.000000,beef
3,746764,0.810000,0.4700,7.920000,carrots
4,747448,0.640000,0.2200,7.630000,strawberries
5,1105314,0.740000,0.2900,23.000000,banana
6,1750339,0.187500,0.2125,14.781700,apple
7,1750347,2.890625,0.3708,4.079375,mushrooms


All amounts are per 100g.

## Export first 10 target food nutrition information

In [173]:
target_whole_foods_df.to_csv("target_ten_whole_food_nutrition_info.csv", index=False)

In [174]:
ten_whole_foods

['apple',
 'banana',
 'beef',
 'blueberries',
 'carrots',
 'chicken_wings',
 'egg',
 'honey',
 'mushrooms',
 'strawberries']